<a href="https://colab.research.google.com/github/Vinh-thuy/data_valorization/blob/main/demo/20230106_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openrouteservice

In [ ]:
#@title Connexion Google Drive {display-mode: "form"}

from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

%cd /content/gdrive/MyDrive/P4T/google_colab/demo/



In [3]:
#@title Construction de la connexion à la base PostgreSQL {display-mode: "form"}

from postgres import postgresQuery
from utilities import DB_HOSTNAME,DB_PORT, DB_NAME, DB_USER, DB_PWD, PG_COL_TO_CLEAN_TABLE_RESTAURANT
pg = postgresQuery(DB_HOSTNAME, DB_PORT, DB_NAME, DB_USER, DB_PWD)

In [4]:
#@title Fonctions Open Route Source {display-mode: "form"}

def list_restaurant_in_request(coordinates, mode_transport):

  import folium
  from folium.plugins import BeautifyIcon
  import pandas as pd
  import openrouteservice as ors
  
  m = folium.Map(location=coordinates[0], tiles='cartodbpositron', zoom_start=13)
 
  # insert your ORS api key
  api_key = '5b3ce3597851110001cf6248dc81b6e4c19a4dd6895edd257e50fe93'
  import openrouteservice as ors
  import folium

  client = ors.Client(key=api_key)

  #listPOI = [[2.7650702987222555, 48.85413769159812],[2.773551801386921, 48.85651435239136],[2.761964837814832, 48.85999968749622],[2.7524732819089146, 48.87563355362418]]

  #coordinates = listPOI

  matrix = client.distance_matrix(
      locations=coordinates,
      profile=mode_transport,
      metrics=['duration', 'distance'],
      destinations=[0],
      validate=False,
      resolve_locations=True
  )

  for marker in coordinates:
      folium.Marker(location=list(reversed(marker))).add_to(m)

  print("Durations in secs: {}\n".format(matrix['durations']))
  print("Distances in m: {}".format(matrix['distances']))

  m
  return matrix['durations'], matrix['distances'], m







def check_ors_distance_bulk(coordinates, mode_transport):

  import folium
  from folium.plugins import BeautifyIcon
  import pandas as pd
  import openrouteservice as ors

  
  # insert your ORS api key
  api_key = '5b3ce3597851110001cf6248dc81b6e4c19a4dd6895edd257e50fe93'
  import openrouteservice as ors
  import folium

  client = ors.Client(key=api_key)

  matrix = client.distance_matrix(
      locations=coordinates,
      profile=mode_transport,
      metrics=['duration', 'distance'],
      destinations=[0],
      validate=False,
      resolve_locations=True
  )

  return matrix['durations'], matrix['distances']








def check_ors_distance(coordinates, mode_transport, distance_max, type_distance):

  import folium
  from folium.plugins import BeautifyIcon
  import pandas as pd
  import openrouteservice as ors
  
  
  # insert your ORS api key
  api_key = '5b3ce3597851110001cf6248dc81b6e4c19a4dd6895edd257e50fe93'
  import openrouteservice as ors
  import folium

  client = ors.Client(key=api_key)


  matrix = client.distance_matrix(
      locations=coordinates,
      profile=mode_transport,
      metrics=['duration', 'distance'],
      destinations=[0],
      validate=False,
      resolve_locations=True
  )

  res = False

  if type_distance == 'duration':
    if matrix['durations'][1][0]/60 < distance_max:
      res = True
  
  if type_distance == 'distance':
    if matrix['distance'][1][0] < distance_max:
      res = True

  #print(coordinates[0])
  #print(coordinates[1])
  #print(matrix['durations'][1][0]/60)
  print("Distance entre : " + str(coordinates[0]) + " et " + str(coordinates[1]) + " = " + str(matrix['durations'][1][0]/60) + " >>>> res = " + str(res))
  print('----------------')

  return res

In [5]:
#@title Récupération de la liste des restaurants de la DB PostgreSQL : liste_restaurant_poi_matrix {display-mode: "form"}

liste_restaurant = pg.query_db("select * from restaurant where adress_city = 'Paris'")
liste_restaurant=liste_restaurant.fillna('NULL') 
liste_restaurant_poi_matrix = liste_restaurant[(liste_restaurant.latitude != 'NULL')][['restaurant_id','name', 'adress_street', 'adress_city', 'latitude', 'longitude', 'openinghours', 'category']]
liste_restaurant_poi_matrix = liste_restaurant_poi_matrix.sort_values(by=['restaurant_id'], ascending=True)

liste_restaurant_poi_matrix.latitude = liste_restaurant_poi_matrix.latitude.astype(float)
liste_restaurant_poi_matrix.longitude = liste_restaurant_poi_matrix.longitude.astype(float)


# On prend à partir de id 324 car l'alimentation de la DB doit etre revue avant cette ID
liste_restaurant_poi_matrix = liste_restaurant_poi_matrix[liste_restaurant_poi_matrix.restaurant_id > 324]

In [6]:
#@title Création d'une position pour la recherche de restaurant >> Geoloc place de l'Opera  loc_matrix / loc_folium {display-mode: "form"}

#Geoloc pour Google ou autre comme Folium
loc_folium = [48.8714368,2.3326198]

#Geoloc pour ORS (matrix)
loc_matrix = [2.3326198, 48.8714368]

In [7]:
#@title Check UNITAIRE pour identifier par la duration de chaque Restaurant par rapport à la Geoloc place de l'Opera {display-mode: "form"}


def check_unitaire_distance(loc_matrix, liste_restaurant, nb_restaurant_check, distance_max, type_transport, critere_distance):


  liste_restaurant = liste_restaurant.sort_values(by=['restaurant_id'], ascending=True)

  liste_restaurant_poi_matrix['check'] = False
  liste_check = liste_restaurant[:nb_restaurant_check].copy()


  for i in range(len(liste_check.restaurant_id)):
    dest = [liste_check['longitude'].iloc[i], liste_check['latitude'].iloc[i]]
    coord = [loc_matrix].copy()
    coord.append(dest)

    #print(dest)
    #print(liste_check[['latitude','longitude']].iloc[i])
    #print(liste_check.restaurant_id.iloc[i])
    #print("-----------------")


    res = liste_check['check'].iloc[i] = check_ors_distance(coord, type_transport, distance_max, critere_distance)

    return res

In [ ]:
#check_unitaire_distance(loc_matrix, liste_restaurant_poi_matrix, 20, 30, 'foot-walking', 'duration')
#check_unitaire_distance(loc_matrix, [2.341321, 48.884422], 20, 30, 'foot-walking', 'duration')

Distance entre : [2.3326198, 48.8714368] et [2.368429, 48.8551576] = 42.4435 >>>> res = False
----------------


False

In [8]:
#@title Calcul Distance entre 2 points {display-mode: "form"}

def calc_distance(loc1, loc2):
  #Geoloc pour ORS (matrix)
  loc_dest = loc2

  coord_check = [loc1].copy()
  coord_check.append(loc_dest)

  #def check_ors_distance(coordinates, mode_transport, distance_max, type_distance):
  return check_ors_distance(coord_check, 'foot-walking', 30, 'duration')

In [ ]:
calc_distance(loc_matrix, [2.341321, 48.884422])

Distance entre : [2.3326198, 48.8714368] et [2.341321, 48.884422] = 22.109166666666667 >>>> res = True
----------------


True

In [49]:
#@title Check BULK pour identifier par la duration de chaque Restaurant par rapport à la Geoloc place de l'Opera {display-mode: "form"}

def check_bulk_distance(loc_matrix, liste_restaurant_init, distance_max, type_transport, critere_distance):
  import pandas as pd

  step = 50

  dfList = []
  for i in range(0, liste_restaurant_init.shape[0], step):
    dfList.append(liste_restaurant_init[i:i+step])


  for i in range(len(dfList)):    
    liste_restaurant = dfList[i]
    
    #nb_restaurant_check += 1
    #liste_restaurant = liste_restaurant.sort_values(by=['restaurant_id'], ascending=True)

    lst_poi = [loc_matrix].copy()

    for lon, lat in zip(liste_restaurant.longitude, liste_restaurant.latitude):
      lst_poi.append([lon, lat])

    mDuration, mDistance = check_ors_distance_bulk(lst_poi, type_transport)


    liste_restaurant_result = liste_restaurant.copy()
    liste_restaurant_result['distance'] = ''
    liste_restaurant_result['result_distance'] = ''
    liste_restaurant_result['coordinate'] = ''



    for (du, di, id) in zip(mDuration[1:], mDistance[1:], liste_restaurant_result[:50].restaurant_id):

      if critere_distance == 'duration':
        liste_restaurant_result.loc[liste_restaurant_result["restaurant_id"] == id, "distance"] = du[0]/60
        if du[0]/60 < distance_max:
            liste_restaurant_result.loc[liste_restaurant_result["restaurant_id"] == id, "result_distance"] = True
        else:
            liste_restaurant_result.loc[liste_restaurant_result["restaurant_id"] == id, "result_distance"] = False
      else:
        liste_restaurant_result.loc[liste_restaurant_result["restaurant_id"] == id, "distance"] = di
        if di[0] < distance_max:
            liste_restaurant_result.loc[liste_restaurant_result["restaurant_id"] == id, "result_distance"] = True
        else:
            liste_restaurant_result.loc[liste_restaurant_result["restaurant_id"] == id, "result_distance"] = False    


    if i == 0:
      result = liste_restaurant_result[liste_restaurant_result.result_distance == True]
    else:
      result = result.append(liste_restaurant_result[liste_restaurant_result.result_distance == True])

  
  return result

In [46]:
#res_bulk_distance = check_bulk_distance(loc_matrix, liste_restaurant_poi_matrix[:50], 30, 'foot-walking', 'duration')
#res_bulk_distance = check_bulk_distance(loc_matrix, liste_restaurant_poi_matrix[:50], 1000, 'foot-walking', 'distance')
#res_bulk_distance 

,restaurant_id,name,adress_street,adress_city,latitude,longitude,openinghours,category,distance,result_distance,coordinate
15,329,Mahabelly,21 Bd Beaumarchais,Paris,48.855158,2.368429,"{'day': 'jeudi,', 'hours': '8h00 to 2h00'}",Restaurant indien,3.551667,True,
98,386,Auberge Nicolas Flamel,51 Rue de Montmorency,Paris,48.863563,2.353103,"[{'day': 'vendredi,', 'hours': '12:15 to 13:15...","['Restaurant gastronomique', 'Restaurant franç...",23.093,True,
100,388,Robert et Louise,64 Rue Vieille-du-Temple,Paris,48.859079,2.359435,"[{'day': 'vendredi,', 'hours': '12:00 to 15:00...","['Restaurant', 'Restaurant français']",14.348667,True,
103,391,Dersou,21 Rue Saint-Nicolas,Paris,48.851399,2.374616,"[{'day': 'vendredi,', 'hours': '19:30 to 22:30...",['Restaurant'],6.246667,True,
105,393,Restaurant Dans le Noir Paris,51 R. Quincampoix,Paris,48.861345,2.350561,"[{'day': 'vendredi,', 'hours': '19:30 to 21:45...","['Restaurant français', 'Boutique de cadeaux',...",22.169333,True,
112,400,Le Saint Sébastien,42 Rue Saint-Sébastien,Paris,48.861524,2.371056,"[{'day': 'vendredi,', 'hours': 'Fermé'}, {'day...","['Restaurant', 'Bar', 'Restaurant français', '...",14.450167,True,
113,401,La Tour Montlhéry Chez Denise,5 Rue des Prouvaires,Paris,48.861534,2.344001,"[{'day': 'vendredi,', 'hours': '12:00 to 00:30...","['Restaurant français', 'Bistro']",25.940333,True,
123,412,Le Rigmarole,10 Rue du Grand Prieuré,Paris,48.865139,2.369111,"[{'day': 'vendredi,', 'hours': '19:00 to 23:00...",['Pizzeria'],19.035167,True,
126,413,La Dame de Pic,20 Rue du Louvre,Paris,48.861466,2.341232,"[{'day': 'vendredi,', 'hours': '12:00 to 13:30...","['Restaurant gastronomique', 'Restaurant franç...",28.0745,True,
129,416,La Mangerie,7 Rue de Jarente,Paris,48.855700,2.362812,"[{'day': 'vendredi,', 'hours': '18:00 to 01:00...","['Cuisine européenne', 'Bar', 'Restaurant espa...",7.923667,True,


In [54]:
liste_restaurant_poi_matrix[liste_restaurant_poi_matrix.name == 'Mahabelly']

,restaurant_id,name,adress_street,adress_city,latitude,longitude,openinghours,category
15,329,Mahabelly,21 Bd Beaumarchais,Paris,48.855158,2.368429,"{'day': 'jeudi,', 'hours': '8h00 to 2h00'}",Restaurant indien


In [10]:
#@title Check horaire restaurant {display-mode: "form"}

import datetime

def DayOfWeekLiteral(dday):
  if dday == 'Monday':
      dayl = "lundi"
  if dday == 'Tuesday':
      dayl = "mardi"
  if dday == 'Wednesday':
      dayl = "mercredi"
  if dday == 'Thursday':
      dayl = "jeudi"
  if dday == 'Friday':
      dayl = "vendredi"
  if dday == 'Saturday':
      dayl = "samedi"  
  if dday == 'Sunday':
      dayl = "dimanche"                             
  return dayl


def check_open(date_select, hour_select, open_plage):
  #parse day and hour from date_select
  date_string = datetime.datetime.strptime(date_select, '%Y-%m-%d')
  day = date_string.strftime('%A')

  if open_plage == 'NULL':
    return False
  
  try:
  #find the day matches open_plage
    for plage in open_plage: 
      if plage['day'].startswith(DayOfWeekLiteral(day)):
        if plage['hours'] != "Fermé": 
          #parse hour
          hours = plage['hours'].replace(" ", "").split(",")
          for hour in hours:
            hour_start = datetime.datetime.strptime(hour.split("to")[0], '%H:%M')
            hour_end = datetime.datetime.strptime(hour.split("to")[1], '%H:%M')
            hour_select_ = datetime.datetime.strptime(hour_select, '%H:%M')
            # decide is in open slot or not
            if (hour_start <= hour_select_) and (hour_select_ <= hour_end):
              return True
  except (ValueError, TypeError):
    return False
  
  return False


In [11]:
#@title Conversion String to Listg pour openningHours : str2List {display-mode: "form"}

def str2List(texte):
  # Python code to demonstrate converting
  # string representation of list to list
  # using ast.literal_eval()
  import ast
  
  # initializing string representation of a list
  ini_list = texte

  if ini_list != 'NULL':
    # Converting string to list
    res = ast.literal_eval(ini_list)

    if type(res) == dict:
      res = [res]
    return res  
  else: 
    return 'NULL'

In [12]:
#@title Check BULK horaire openning {display-mode: "form"}

def check_bulk_openning(liste_restaurant, date_select, hour_select):

  liste_restaurant['check_open'] = False
  liste_restaurant = liste_restaurant.sort_values(by=['restaurant_id'], ascending=True)

  for i in range(len(liste_restaurant)):

    res_open = check_open(date_select, hour_select, str2List(liste_restaurant.iloc[i].openinghours))

    if res_open:
        liste_restaurant['check_open'].iloc[i] = True
    else:
        liste_restaurant['check_open'].iloc[i] = False

  return liste_restaurant[liste_restaurant.check_open == True]

In [ ]:
#res_bulk_distance_horaire = check_bulk_openning(res_bulk_distance, '2023-01-10','19:59')
#res_bulk_distance_horaire

In [13]:
#@title Check BULK category {display-mode: "form"}

def check_bulk_category(liste_restaurant, category):

  from re import search
  
  liste_restaurant['check_category'] = False
  liste_restaurant = liste_restaurant.sort_values(by=['restaurant_id'], ascending=True)

  res_category = False
  for i, elt in enumerate(res_bulk_distance_horaire.category):
    if search(category, elt):
      res_category = True
    else:
      res_category = False
    

    if res_category:
        liste_restaurant['check_category'].iloc[i] = True
    else:
        liste_restaurant['check_category'].iloc[i] = False

  return liste_restaurant[liste_restaurant.check_category == True]
  

In [ ]:
#res_bulk_distance_horaire_category = check_bulk_category(res_bulk_distance_horaire, 'Restaurant gastronomique')
#res_bulk_distance_horaire_category

In [59]:
#@title Show Map Folium {display-mode: "form"}

def show_map(liste_restaurant, loc_folium, type_distance):
  import folium


  #liste_restaurant = res_bulk_distance_horaire_category.copy()


  lst_folium = []
  res_bulk_distance_horaire = liste_restaurant.sort_values(by=['restaurant_id'], ascending=True)

  for lon, lat in zip(liste_restaurant.longitude, liste_restaurant.latitude):
    lst_folium.append([lat, lon])

  lst_folium




  m = folium.Map(location=loc_folium, tiles='cartodbpositron', zoom_start=15)

  # Some coordinates in Berlin
  coordinates = lst_folium

  if type_distance == 'duration':
    metrique = 'minutes'
  else:
    metrique = 'metres'

  for i, marker in enumerate(lst_folium):
      
      test = folium.Html("text", script=True)
      icon = folium.Icon(color='red', icon='info-sign')
      label = liste_restaurant.name.iloc[i] + " -- distance : " + str(round(liste_restaurant.distance.iloc[i])) + " ("+ metrique + ")"
      popup = folium.Popup(label, max_width=300,min_width=300)
      folium.Marker(location=list(marker), popup=popup, icon=icon).add_to(m)

  marker=folium.Marker(loc_folium,popup='Position de la recherche',icon = folium.Icon(color='blue', icon='info-sign'))

  marker.add_to(m)
  m

  return m

In [64]:
nb_resto = 9999999                           # variable pour limiter les calcules

type_distance = 'duration'              # duration (minute) ou distance (metre)
distance = 10                           # duration (minute) ou distance (metre)
type_transport = 'foot-walking'         # 'foot-walking' ; 'driving-car'; 'wheelchair' ; ...
date = '2023-01-10'                     # date souhaitée pour le restaurant
heure = '19:59'                         # heure souhaitée pour le restaurant
category  = 'Restaurant italien'  # catégorie du restautant
loc_folium = [48.8530187,2.3689949]     # Geoloc de la position de recherche
loc_matrix = loc_folium[::-1]           # Geoloc inversé pour le calcul Matrix de ORS



# Filtrage sur la distance en minutes ou metres
res_bulk_distance = check_bulk_distance(loc_matrix, liste_restaurant_poi_matrix, distance, type_transport, type_distance)

# Filtrage sur les horaires d'ouverture
res_bulk_distance_horaire = check_bulk_openning(res_bulk_distance, date,heure)

# Filtrage sur la catégorie du restaurant
res_bulk_distance_horaire_category = check_bulk_category(res_bulk_distance_horaire, category)

# Affichage de la carte avec les résultats des 3 filtres
m = show_map(res_bulk_distance_horaire_category, loc_folium, type_distance)
m

In [65]:
res_bulk_distance_horaire_category

,restaurant_id,name,adress_street,adress_city,latitude,longitude,openinghours,category,distance,result_distance,coordinate,check_open,check_category
295,694,0039 Ristorante Italiano Il Toscano,24 Rue des Tournelles,Paris,48.854949,2.367379,"[{'day': 'lundi,', 'hours': 'Fermé'}, {'day': ...","['Restaurant italien', 'Restaurant familial', ...",3.776167,True,,True,True
340,744,Salento Marais,6 Rue de Jarente,Paris,48.855741,2.362980,"[{'day': 'lundi,', 'hours': '12:00 to 14:30, 1...",['Restaurant italien'],7.812333,True,,True,True
370,774,Le Petit Italien,5 Rue Saint-Gilles,Paris,48.857816,2.366271,"[{'day': 'lundi,', 'hours': '12:00 to 15:00, 1...",['Restaurant italien'],8.4195,True,,True,True
379,781,Amici Miei,44 Rue St Sabin,Paris,48.858186,2.369364,"[{'day': 'lundi,', 'hours': 'Fermé'}, {'day': ...","['Restaurant italien', 'Pizzeria', 'Pizzas à e...",8.176667,True,,True,True
389,790,Auberge de Venise Bastille,2 Rue de la Bastille,Paris,48.853771,2.367493,"[{'day': 'lundi,', 'hours': '10:00 to 23:00'},...",['Restaurant italien'],2.217,True,,True,True
391,792,The Brooklyn Pizzeria,33 Bd Beaumarchais,Paris,48.855882,2.368202,"[{'day': 'lundi,', 'hours': '11:30 to 23:30'},...","['Pizzeria', 'Restaurant italien', 'Restaurant']",4.719667,True,,True,True
395,795,Le Rusti,8 Rue des Tournelles,Paris,48.854186,2.367439,"[{'day': 'lundi,', 'hours': '12:00 to 14:30, 1...","['Restaurant italien', 'Bistro', 'Restaurant']",2.744333,True,,True,True
405,805,Assaporare 10 sur 10,7/9 Rue Saint-Nicolas,Paris,48.850941,2.374265,"[{'day': 'lundi,', 'hours': 'Fermé'}, {'day': ...",['Restaurant italien'],7.7,True,,True,True
418,819,Rivoluzione,24 Rue des Taillandiers,Paris,48.855351,2.374829,"[{'day': 'lundi,', 'hours': '12:00 to 14:30, 1...","['Restaurant italien', 'Pizzeria']",6.815667,True,,True,True
432,833,Piadineria 14 07,13 Rue St Sabin,Paris,48.855317,2.371818,"[{'day': 'lundi,', 'hours': 'Fermé'}, {'day': ...",['Restaurant italien'],4.813833,True,,True,True


In [67]:
res_bulk_distance_horaire_category.to_csv('data.csv')
!cp data.csv "/content/gdrive/MyDrive/P4T/google_colab/demo/"

cp: 'data.csv' and '/content/gdrive/MyDrive/P4T/google_colab/demo/data.csv' are the same file


In [68]:
!ls

data.csv  local_fonction.py  postgres.py  __pycache__  utilities.py


In [ ]:
#pip install OSMPythonTools
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()

heidelberg = nominatim.query(44.8342211378199, -0.5790779109885305, reverse=True, zoom=10)


heidelberg.address()


{'city': 'Bordeaux',
 'municipality': 'Bordeaux',
 'county': 'Gironde',
 'ISO3166-2-lvl6': 'FR-33',
 'state': 'Nouvelle-Aquitaine',
 'ISO3166-2-lvl4': 'FR-NAQ',
 'region': 'France métropolitaine',
 'country': 'France',
 'country_code': 'fr'}

In [ ]:
heidelberg.address()

{'town': 'Montévrain',
 'municipality': 'Torcy',
 'county': 'Seine-et-Marne',
 'ISO3166-2-lvl6': 'FR-77',
 'state': 'Île-de-France',
 'ISO3166-2-lvl4': 'FR-IDF',
 'region': 'France métropolitaine',
 'postcode': '77144',
 'country': 'France',
 'country_code': 'fr'}

In [18]:
len(liste_restaurant_poi_matrix)

531